# Notebook Title

## Setup Python and R environment
you can ignore this section

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

Durante la inicialización - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 


In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')
require('tidycensus')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse
Loading required package: tidycensus


## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [4]:
%%R
# Read the CSV file with custom delimiters
df <- read.csv('Prueba1.csv')
df

                                                               Name.STATE.COUNTY.TRACT.GEOID
1                                              100% Playground;36.0;47.0;97400.0;364797400.0
2                                          174th Street Playground;36.0;5.0;6200.0;3656200.0
3                                          227 Street Playground;36.0;5.0;40600.0;36540600.0
4                                                  Abe Lincoln;36.0;61.0;16900.0;366116900.0
5                                               Abigail Playground;36.0;5.0;7900.0;3657900.0
6                                       Agnes Haywood Playground;36.0;5.0;39000.0;36539000.0
7                                               Albemarle Park;36.0;47.0;49800.0;364749800.0
8                                  Albert J. Parham Playground;36.0;47.0;31300.0;364731300.0
9  

                              Alexander Hamilton Playground;36.0;61.0;25900.0;366125900.0
10                                          Alfred E. Smith Park;36.0;61.0;2500.0;36612500.0
11                                     Alice Kornegay Triangle;36.0;61.0;24200.0;366124200.0
12                                           Allerton Playground;36.0;5.0;31800.0;36531800.0
13                                       Alley Park (PS 213);36.0;81.0;137700.0;3681137700.0
14                                           Alley Pond Park;36.0;81.0;128300.0;3681128300.0
15                    Alley Pond Park (Alley Athletic Field);36.0;81.0;156700.0;3681156700.0
16                       Alley Pond Park (Alley Pond Spring);36.0;81.0;137700.0;3681137700.0
17                      Alley Pond Park (Horatio Playground);36.0;81.0;138501.0;3681138501.0
18                                          Alstyne Playground;36.0;81.0;41300.0;368141300.0
19                                         American Playground;36.0;47.0;

In [6]:
%%R
# Check column names
colnames(df)





UsageError: Cell magic `%%R` not found.


## 👉 Grab Census Data

1. loading the Census API key

In [5]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()

CENSUS_API_KEY = "a435d72f84653a5895173d187536a34d55ac0afc"

In [6]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("a435d72f84653a5895173d187536a34d55ac0afc"))

To install your API key for use in future sessions, run this function with `install = TRUE`.


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


In [7]:
%%R 

# Finding the Census Varaibles for the ACS 5 year survey
# Generall you'd do this in CensusReporter, but since it's down sometimes, here it is using tidycensus's load_variables function

# get every single variable in the ACS5
all_census_vars <- load_variables(2021, "acs5", cache = TRUE) 

filtered_census_vars <- all_census_vars %>% 
    filter(grepl("median income", label, ignore.case = TRUE))   # filter to those containing "median income"
    
# write to CSV so we can view it in python
filtered_census_vars %>% 
    write_csv("filtered_census_vars.csv")

# show the first few rows
filtered_census_vars %>%
    select(-geography) %>% # remove the geography column
    print(n = 20) # print the first 20 rows

# A tibble: 46 × 3
   name         label                                                    concept
   <chr>        <chr>                                                    <chr>  
 1 B06011PR_001 Estimate!!Median income in the past 12 months --!!Total: MEDIAN…
 2 B06011PR_002 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 3 B06011PR_003 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 4 B06011PR_004 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 5 B06011PR_005 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 6 B06011_001   Estimate!!Median income in the past 12 months --!!Total: MEDIAN…
 7 B06011_002   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 8 B06011_003   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 9 B06011_004   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
10 B06011_005   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
11 B07011

In [26]:
%%R -o nyc_census_data
# the variable B01003_001E was selectd from the census table 
# for population, which we found in censusreporter here:
# https://censusreporter.org/tables/B01003/

# in the table below, pick the geography, the variables, and the survey you want to pull from
# see the possible values here https://walker-data.com/tidycensus/articles/basic-usage.html

# Get variable from ACS
nyc_census_data <- get_acs(geography = "tract", 
                      state='NY',
                      county = c("New York", "Kings", "Queens", "Bronx", "Richmond"),
                      variables = c(
                        population="B01003_001",
                        med_inc="B19013_001",
                        poverty="B17001_001"
                      ), 
                      year = 2021,
                      survey="acs5",
                      geometry=F)

nyc_census_data

# A tibble: 6,981 × 5
   GEOID       NAME                                    variable   estimate   moe
   <chr>       <chr>                                   <chr>         <dbl> <dbl>
 1 36005000100 Census Tract 1, Bronx County, New York  population     6661   702
 2 36005000100 Census Tract 1, Bronx County, New York  poverty           0    18
 3 36005000100 Census Tract 1, Bronx County, New York  med_inc          NA    NA
 4 36005000200 Census Tract 2, Bronx County, New York  population     4453   563
 5 36005000200 Census Tract 2, Bronx County, New York  poverty        4453   563
 6 36005000200 Census Tract 2, Bronx County, New York  med_inc       70867 25423
 7 36005000400 Census Tract 4, Bronx County, New York  population     6000   903
 8 36005000400 Census Tract 4, Bronx County, New York  poverty        6000   903
 9 36005000400 Census Tract 4, Bronx County, New York  med_inc       98090 18180
10 36005001600 Census Tract 16, Bronx County, New York population     6038   665
# ℹ 6,

Getting data from the 2017-2021 5-year ACS
Using FIPS code '36' for state 'NY'
Using FIPS code '061' for 'New York County'
Using FIPS code '047' for 'Kings County'
Using FIPS code '081' for 'Queens County'
Using FIPS code '005' for 'Bronx County'
Using FIPS code '085' for 'Richmond County'


In [27]:
nyc_census_data

,GEOID,NAME,variable,estimate,moe
1,36005000100,"Census Tract 1, Bronx County, New York",population,6661.0,702.0
2,36005000100,"Census Tract 1, Bronx County, New York",poverty,0.0,18.0
3,36005000100,"Census Tract 1, Bronx County, New York",med_inc,NaN,NaN
4,36005000200,"Census Tract 2, Bronx County, New York",population,4453.0,563.0
5,36005000200,"Census Tract 2, Bronx County, New York",poverty,4453.0,563.0
...,...,...,...,...,...
6977,36085032300,"Census Tract 323, Richmond County, New York",poverty,1105.0,304.0
6978,36085032300,"Census Tract 323, Richmond County, New York",med_inc,86471.0,25095.0
6979,36085990100,"Census Tract 9901, Richmond County, New York",population,0.0,13.0
6980,36085990100,"Census Tract 9901, Richmond County, New York",poverty,0.0,13.0


In [28]:
Toilet_df = pd.read_excel('ToiletWithGeos.xlsx', dtype={'GEOID': str}) 
Toilet_df['GEOID'] = Toilet_df['GEOID'].astype(str).str[:11]

Toilet_df

,Name,Location,Open Year-Round,Handicap Accessible,Borough,Comments,Latitude,Longitude,GEOID,STATE,COUNTY,TRACT,BLOCK
0,100% Playground,"Glenwood Road, East 100 & East 101 streets",Yes,NaN,Brooklyn,NaN,40.646908,-73.899467,36047097400,36,47,97400,2002
1,174th Street Playground,"East 174 Street, Stratford Avenue, Bronx River...",Yes,NaN,Bronx,NaN,40.833885,-73.876596,36005006200,36,5,6200,3000
2,227 Street Playground,Bronx Boulevard between East 226 and East 228 ...,Yes,Yes,Bronx,NaN,40.888553,-73.855013,36005040600,36,5,40600,1001
3,Abe Lincoln,"East 135 Street, between Madison & 5 avenues",Yes,NaN,Manhattan,NaN,40.783060,-73.971249,36061016900,36,61,16900,6000
4,Abigail Playground,"East 156 Street, Tinton Avenue",No,NaN,Bronx,NaN,40.817880,-73.905006,36005007900,36,5,7900,1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,Wingate Park,Brooklyn Avenue & Rutland Road,Yes,NaN,Brooklyn,NaN,40.659723,-73.945358,36047081000,36,47,81000,1004
612,Woods Playground,Bergen Street & Utica Avenue,Yes,NaN,Brooklyn,NaN,40.675045,-73.930540,36047030900,36,47,30900,2000
613,Woodtree Playground,"20 Avenue, 37 Street, 38 Street",Yes,NaN,Queens,NaN,40.761101,-73.919665,36081006100,36,81,6100,4001
614,Yak Playground,Avenue Y between Coyle & Batchelder streets,Yes,Yes,Brooklyn,NaN,40.595089,-73.937972,36047057200,36,47,57200,2001


In [22]:
nyc_census_data[nyc_census_data['GEOID'] == '36047097400']

,GEOID,NAME,variable,estimate,moe
3232,36047097400,"Census Tract 974, Kings County, New York",population,2792.0,536.0
3233,36047097400,"Census Tract 974, Kings County, New York",poverty,2464.0,417.0
3234,36047097400,"Census Tract 974, Kings County, New York",med_inc,76767.0,17732.0


In [32]:
merge = Toilet_df.merge(nyc_census_data, left_on='GEOID', right_on='GEOID')
merge

merge.to_excel('merged.xlsx', index=False)

In [ ]:
%%R
#show me TRACT in the data


Error in `select()`:
! Can't subset columns that don't exist.
✖ Column `tract` doesn't exist.
Run `rlang::last_trace()` to see where the error occurred.



Error in select(., tract) :


RInterpreterError: Failed to parse and evaluate line '#show me TRACT in the data\nnyc_census_data %>%\n    select(tract) %>%\n    print(n=5)\n'.
R error message: 'Error in select(., tract) :'

In [ ]:
%%R
#turn nyc_census_data as a csv file
write.csv(nyc_census_data, "nyc_census_data2.csv")

In [ ]:
%%R 
nyc_census_data


Error in library(tmap) : there is no package called ‘tmap’


RInterpreterError: Failed to parse and evaluate line 'nyc_census_data\n#make a map of the data\nlibrary(tmap)\n'.
R error message: 'Error in library(tmap) : there is no package called ‘tmap’'

## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [ ]:
%%R 
#df <- left_join(df, nyc_census_data, by==c("GEOID"=="geoid"))

#pivot the census data from long to wide format 
nyc_census_data_wide <- nyc_census_data %>%
  pivot_wider(names_from = "GEOID", values_from = "GEOID")

nyc_census_data_wide



Simple feature collection with 6981 features and 2331 fields (with 3 geometries empty)
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -74.25609 ymin: 40.4961 xmax: -73.70036 ymax: 40.91771
Geodetic CRS:  NAD83
# A tibble: 6,981 × 2,332
   NAME          variable estimate   moe                  geometry `36081014700`
   <chr>         <chr>       <dbl> <dbl>        <MULTIPOLYGON [°]> <chr>        
 1 Census Tract… populat…     2863   513 (((-73.9137 40.76548, -7… 36081014700  
 2 Census Tract… poverty      2862   513 (((-73.9137 40.76548, -7… 36081014700  
 3 Census Tract… med_inc     71815 18034 (((-73.9137 40.76548, -7… 36081014700  
 4 Census Tract… populat…     3655   529 (((-73.96103 40.59616, -… <NA>         
 5 Census Tract… poverty      3655   529 (((-73.96103 40.59616, -… <NA>         
 6 Census Tract… med_inc     67315 10294 (((-73.96103 40.59616, -… <NA>         
 7 Census Tract… populat…     2450   372 (((-74.02064 40.73039, -… <NA>         
 8 Census Tract

In [ ]:
%%R

nyc_census_data

#download the data as a csv file
write.csv(nyc_census_data, "census_data1.csv")

In [ ]:
%%R
#DOWNLOAD nyc_census_data_wide as a     CSV file
write.csv(nyc_census_data_wide, "nyc_census_data_wide.csv")


In [ ]:
%%R
#define with df the csv 'ToiletWithGeos.csv'
df <- read_csv('ToiletWithGeos (2).csv')

df


New names:
• `SUFFIX` -> `SUFFIX...9`
• `POP100` -> `POP100...10`
• `GEOID` -> `GEOID...11`
• `CENTLAT` -> `CENTLAT...12`
• `BLOCK` -> `BLOCK...13`
• `AREAWATER` -> `AREAWATER...14`
• `STATE` -> `STATE...15`
• `BASENAME` -> `BASENAME...16`
• `OID` -> `OID...17`
• `LSADC` -> `LSADC...18`
• `INTPTLAT` -> `INTPTLAT...19`
• `FUNCSTAT` -> `FUNCSTAT...20`
• `NAME` -> `NAME...21`
• `OBJECTID` -> `OBJECTID...22`
• `TRACT` -> `TRACT...23`
• `CENTLON` -> `CENTLON...24`
• `BLKGRP` -> `BLKGRP...25`
• `AREALAND` -> `AREALAND...26`
• `HU100` -> `HU100...27`
• `INTPTLON` -> `INTPTLON...28`
• `MTFCC` -> `MTFCC...29`
• `LWBLKTYP` -> `LWBLKTYP...30`
• `UR` -> `UR...31`
• `COUNTY` -> `COUNTY...32`
• `SUFFIX` -> `SUFFIX...33`
• `POP100` -> `POP100...34`
• `GEOID` -> `GEOID...35`
• `CENTLAT` -> `CENTLAT...36`
• `BLOCK` -> `BLOCK...37`
• `AREAWATER` -> `AREAWATER...38`
• `STATE` -> `STATE...39`
• `BASENAME` -> `BASENAME...40`
• `OID` -> `OID...41`
• `LSADC` -> `LSADC...42`
• `INTPTLAT` -> `INTPTLAT...43`
• 

NULL


Además: Warning message:
Unknown or uninitialised column: `GEOID`. 


NULL


Además: Warning message:
Unknown or uninitialised column: `TRACT`. 


In [ ]:
%%R

library(dplyr)

# Print the column names of df and nyc_census_data to verify
print(colnames(df))
print(colnames(nyc_census_data))

# Verify the data types of the columns
print(class(df$GEOID))
print(class(nyc_census_data$geoid))

# Try converting the data types if necessary
# df$GEOID <- as.character(df$GEOID)
# nyc_census_data$geoid <- as.character(nyc_census_data$geoid)

# Attempt the left join operation again
result <- left_join(df, nyc_census_data, by = c("GEOID" = "geoid"))


 [1] "Name"                "Location"            "Open Year-Round"    
 [4] "Handicap Accessible" "Borough"             "Comments"           
 [7] "Latitude"            "Longitude"           "SUFFIX...9"         
[10] "POP100...10"         "GEOID...11"          "CENTLAT...12"       
[13] "BLOCK...13"          "AREAWATER...14"      "STATE...15"         
[16] "BASENAME...16"       "OID...17"            "LSADC...18"         
[19] "INTPTLAT...19"       "FUNCSTAT...20"       "NAME...21"          
[22] "OBJECTID...22"       "TRACT...23"          "CENTLON...24"       
[25] "BLKGRP...25"         "AREALAND...26"       "HU100...27"         
[28] "INTPTLON...28"       "MTFCC...29"          "LWBLKTYP...30"      
[31] "UR...31"             "COUNTY...32"         "SUFFIX...33"        
[34] "POP100...34"         "GEOID...35"          "CENTLAT...36"       
[37] "BLOCK...37"          "AREAWATER...38"      "STATE...39"         
[40] "BASENAME...40"       "OID...41"            "LSADC...42"         
[43] "

RInterpreterError: Failed to parse and evaluate line '\nlibrary(dplyr)\n\n# Print the column names of df and nyc_census_data to verify\nprint(colnames(df))\nprint(colnames(nyc_census_data))\n\n# Verify the data types of the columns\nprint(class(df$GEOID))\nprint(class(nyc_census_data$geoid))\n\n# Try converting the data types if necessary\n# df$GEOID <- as.character(df$GEOID)\n# nyc_census_data$geoid <- as.character(nyc_census_data$geoid)\n\n# Attempt the left join operation again\nresult <- left_join(df, nyc_census_data, by = c("GEOID" = "geoid"))\n'.
R error message: 'Error in left_join(df, nyc_census_data, by = c(GEOID = "geoid")) : \n✖ Problem with `GEOID`.'
R stdout:
Warning message:
Unknown or uninitialised column: `GEOID`.

In [ ]:
%%R
library(readr)
library(dplyr)

# Read the original ToiletWithGeos (2) CSV file
toilet_df <- read_csv("ToiletWithGeos (2).csv")

# Modify "COUNTY" column as done previously
toilet_df <- toilet_df %>%
  mutate(COUNTY = if_else(COUNTY == 5, sprintf("%03d", COUNTY), sprintf("%02d", COUNTY)))

# Ensure "TRACT" is six digits
toilet_df <- toilet_df %>%
  mutate(TRACT = sprintf("%06d", TRACT))

# Create "GEOIDCORRECT" by merging "STATE", "COUNTY", and "TRACT"
toilet_df <- toilet_df %>%
  mutate(GEOIDCORRECT = paste0(STATE, COUNTY, TRACT))

# Read the cleaned census_data CSV file (assuming it has been cleaned as described)
census_df <- read_csv("path/to/your/cleaned_census_data.csv", col_types = cols(
  GEOIDCORRECT = col_character(),
  NAME = col_character(),
  variable = col_character(),
  estimate = col_double()
))

# Merge the two data frames on "GEOIDCORRECT"
merged_df <- merge(toilet_df, census_df, by = "GEOIDCORRECT")

# Write the merged DataFrame to a new CSV file
write_csv(merged_df, "Final_Merged_Dataset.csv")

# Note: Replace "path/to/your/" with the actual file paths for your datasets


New names:
• `SUFFIX` -> `SUFFIX...9`
• `POP100` -> `POP100...10`
• `GEOID` -> `GEOID...11`
• `CENTLAT` -> `CENTLAT...12`
• `BLOCK` -> `BLOCK...13`
• `AREAWATER` -> `AREAWATER...14`
• `STATE` -> `STATE...15`
• `BASENAME` -> `BASENAME...16`
• `OID` -> `OID...17`
• `LSADC` -> `LSADC...18`
• `INTPTLAT` -> `INTPTLAT...19`
• `FUNCSTAT` -> `FUNCSTAT...20`
• `NAME` -> `NAME...21`
• `OBJECTID` -> `OBJECTID...22`
• `TRACT` -> `TRACT...23`
• `CENTLON` -> `CENTLON...24`
• `BLKGRP` -> `BLKGRP...25`
• `AREALAND` -> `AREALAND...26`
• `HU100` -> `HU100...27`
• `INTPTLON` -> `INTPTLON...28`
• `MTFCC` -> `MTFCC...29`
• `LWBLKTYP` -> `LWBLKTYP...30`
• `UR` -> `UR...31`
• `COUNTY` -> `COUNTY...32`
• `SUFFIX` -> `SUFFIX...33`
• `POP100` -> `POP100...34`
• `GEOID` -> `GEOID...35`
• `CENTLAT` -> `CENTLAT...36`
• `BLOCK` -> `BLOCK...37`
• `AREAWATER` -> `AREAWATER...38`
• `STATE` -> `STATE...39`
• `BASENAME` -> `BASENAME...40`
• `OID` -> `OID...41`
• `LSADC` -> `LSADC...42`
• `INTPTLAT` -> `INTPTLAT...43`
• 

RInterpreterError: Failed to parse and evaluate line 'library(readr)\nlibrary(dplyr)\n\n# Read the original ToiletWithGeos (2) CSV file\ntoilet_df <- read_csv("ToiletWithGeos (2).csv")\n\n# Modify "COUNTY" column as done previously\ntoilet_df <- toilet_df %>%\n  mutate(COUNTY = if_else(COUNTY == 5, sprintf("%03d", COUNTY), sprintf("%02d", COUNTY)))\n\n# Ensure "TRACT" is six digits\ntoilet_df <- toilet_df %>%\n  mutate(TRACT = sprintf("%06d", TRACT))\n\n# Create "GEOIDCORRECT" by merging "STATE", "COUNTY", and "TRACT"\ntoilet_df <- toilet_df %>%\n  mutate(GEOIDCORRECT = paste0(STATE, COUNTY, TRACT))\n\n# Read the cleaned census_data CSV file (assuming it has been cleaned as described)\ncensus_df <- read_csv("path/to/your/cleaned_census_data.csv", col_types = cols(\n  GEOIDCORRECT = col_character(),\n  NAME = col_character(),\n  variable = col_character(),\n  estimate = col_double()\n))\n\n# Merge the two data frames on "GEOIDCORRECT"\nmerged_df <- merge(toilet_df, census_df, by = "GEOI